In [ ]:
import json


In [ ]:
# load everything from json, at least 2GB of ram is needed for this
game = {}
# all keys inside game params
keys = {}
with open('tools/GameParams.json', 'r') as j:
    game = json.load(j)
    keys = game.keys()
    # just to make sure something is printed and everything are read
    print(len(keys))

ships = {}
with open('tools/ship_all.json', 'r') as j:
    ships = json.load(j)
    print(len(ships))


In [ ]:
# write json object to disk
def writeToJson(json_dict, name):
    with open(name, 'w+') as l:
        l.write(json.dumps(json_dict))

In [ ]:
# find keys in object, can be improved to go deep
def findKey(name, keys):
    temp = []
    for key in keys:
        if name.lower() in key.lower():
            temp.append(key)
    return temp

In [39]:
# find the first object from the dict
def findFirstObject(name, dic, rootMode=False):
    if isinstance(dic, dict):
        for key in dic.keys():
            temp = dic[key]
            # check if this is the object we want
            if name == key:
                if rootMode is True:
                    return dic
                else:
                    return temp
            result = findFirstObject(name, temp, rootMode)
            if result != None:
                return result
    elif isinstance(dic, list):
        # go through the list
        for value in dic:
            result = findFirstObject(name, value, rootMode)
            if result != None:
                return result

In [ ]:
# Find unique update and their ship
unique_upgrades = findKey('Special_Mod', keys)
# this adds ship id to legendary upgrade
legendary = {}
# this add legendary upgrades to ship
ship_upgrade = {}
# get their id and the ship they belong to
for upgrade in unique_upgrades:
    temp = game[upgrade]
    upgrade_id = temp['id']
    upgrade_id_str = str(upgrade_id)
    # remember to add slot later
    slot = temp['slot']
    
    legendary[upgrade_id_str] = []
    for ship in temp['ships']:
        ship_id = game[ship]['id']
        ship_id_str = str(ship_id)
        
        # init
        if not ship_id_str in ship:
            ship_upgrade[ship_id_str] = {"legendary_upgrades": []}

        # you need to the slot as well
        ship_upgrade[ship_id_str]["legendary_upgrades"].append({"upgrade_id": upgrade_id_str, "slot": slot})
        legendary[upgrade_id_str].append({"ship_id": ship_id, "slot": slot})

writeToJson(legendary, 'legendary_upgrade.json')
writeToJson(ship_upgrade, 'ship_legendary_upgrade.json')


In [ ]:
findKey('venezia', keys)

In [ ]:
writeToJson(game['PISC110_Venezia'], 'venezia.json')

In [ ]:
game['PCEM004_Halloween19_10lvl']

In [40]:
def retrieveDataFromShip(ship):
    temp = {}
    # extract permo flages
    permoflages = ship['permoflages'].copy()
    nativePermoflage = ship['nativePermoflage']
    permoflages.append(nativePermoflage)
    # we need to get all permoflage_id
    permo_ids = []
    for permo in permoflages:
        if permo != '':
            permo_ids.append(game[permo]['id'])
    # add it to temp
    temp['permoflages'] = permo_ids

    # extract HE penetration
    artillery = findFirstObject('HitLocationArtillery', ship, rootMode=True)
    if artillery != None:
        ammoList = artillery['ammoList']
        he = [x for x in ammoList if '_HE' in x]
        # some ships doesn't have HE
        if len(he) > 0:
            # add it to temp
            temp['alphaPiercingHE'] = game[he[0]]['alphaPiercingHE']

    # get sigma
    sigma = findFirstObject('ammoPool', ship, rootMode=True)
    if sigma != None:
        temp['sigma'] = sigma['sigmaCount']
        print(ship['name'])
        print(temp['sigma'])
    return temp

In [41]:
# loop through all ship 
additional_ship_info = {}
for ship in ships:
    ship_str = ships[ship]['ship_id_str']
    # it should only has one
    additional_ship_info[ship] = retrieveDataFromShip(game[findKey(ship_str, keys)[0]])
writeToJson(additional_ship_info, 'ship_additional.json')


PASC510_Alaska
2.05
PJSB510_Shikishima
1.0
PZSB509_Izumo_Bajie
1.0
PASC599_Black_Alaska
2.05
PRSC910_Pr_66_Moskva
2.05
PRSC525_Kirov
1.0
PRSC610_Smolensk
2.0
PWSD510_Friesland
2.0
PJSD998_Arashi
2.0
PJSC934_Zao_1944
1.0
PJSC520_Yoshino
1.0
PJSB509_Musashi
1.8
PASC707_Flint
1.7
PBSA518_Ark_Royal
1.0
PRSB518_Lenin
1.7
PRSC513_Varyag
2.0
PJSC709_Haguro
1.0
PJSD706_Shinonome
2.0
PJSC727_Blue_Dragon
1.0
PBSD517_Cossack
2.0
PRSC528_Ochakov
2.0
PJSB918_Yamato_1944
1.0
PJSC598_Black_Atago
1.0
PBSC516_London
1.0
PFSB518_Jean_Bart
1.0
PGSD508_Z_39
2.0
PGSC706_HSF_Graf_Spee
1.9
PJSA518_Kaga
1.0
PJSD912_Shimakaze_1943
2.0
PBSB910_Conqueror
1.9
PASB705_Texas_1944
1.8
PRSC606_Admiral_Makarov
2.0
PJSC708_ARP_Takao
1.0
PASB510_Ohio
1.0
PGSB598_Black_Tirpitz
1.0
PJSD518_Asashio
2.0
PRSC509_Kronshtadt
2.05
PWSD910_Lappland
2.0
PJSD510_Hayate
2.0
PJSC707_Ashigara
1.0
PASB509_Missouri
1.9
PGSA598_Black_Graf_Zeppelin
1.0
PJSC737_Nachi
1.0
PISD510_Paolo_Emilio
2.0
PJSB706_Kirishima
1.0
PJSC510_Azumaya
1.0
P